In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [1]:
import logging
import time

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# read-in cleaned data and parse to corpus and tokenizer
# path = '../data/'
path = '/content/'
filepath = path + 'allphishsets.csv'

df = pd.read_csv(filepath)
df = df.sort_values(by=['showdate', 'set', 'position'],
                    ascending=[True, True, True])

df.loc[df['times_played'] <= 2, 'slug'] = 'wildcard'
df.loc[df['times_played'] <= 2, 'times_played'] = 510

songstring = df[['showdate', 'set', 'slug']].groupby(['showdate', 'set'])['slug']\
                                            .apply(lambda x: '|'.join(x)).reset_index()

songstring['full'] = songstring.apply(lambda row: f"set-{row['set']}|{row['slug']}", axis=1)

songstring = songstring[['showdate', 'full']].groupby(['showdate'])['full']\
                                             .apply(lambda x: '|'.join(x)).reset_index()

songstring['full'] += '|eos'

songstring = songstring[songstring['showdate']>'1990-01-01']

corpus = [''.join(map(lambda s: s.replace('|', ' '), f))
          for f in songstring['full']]

corpus = ' '.join(corpus).split(' ')

def PrepareDataset(corpus: list, n: int,
                   batch_size: int, train_split: float):
    """
    Prepares Datasets for training and validation data from Setlist data
    Args:
      corpus :: list :: full corpus of songs composed of song sequences
      n :: int :: sequence length to trim
      batch_size :: int :: batch size for datasets
      train_split :: float :: values between 0 and 1, splits the data for
                              training and validation
    """
    texts = []
    for i in range(len(corpus)-n):
        texts.append(' '.join(corpus[i:i+n]))

    texts = texts[::n]

    tokenizer = Tokenizer(filters='')
    tokenizer.fit_on_texts(texts)

    x_inputs = []
    x_outputs = []
    for line in texts:
        token_list = tokenizer.texts_to_sequences([line])[0]
        x_inputs.append(token_list[:-1])  #drop last song
        x_outputs.append(token_list[1:])  #drop first song

    x_inputs = np.array(x_inputs)
    x_outputs = np.array(x_outputs)

    buffer_size = len(x_inputs)
    train_size = int(train_split*buffer_size)

    dataset = tf.data.Dataset.from_tensor_slices((x_inputs, x_outputs))
    shuffled_data = dataset.shuffle(buffer_size)

    train_data = dataset.take(train_size) \
                        .batch(batch_size) \
                        .prefetch(buffer_size=tf.data.AUTOTUNE)

    val_data = dataset.skip(train_size) \
                      .batch(batch_size) \
                      .prefetch(buffer_size=tf.data.AUTOTUNE)

    return train_data, val_data, tokenizer

In [2]:
# https://www.tensorflow.org/text/tutorials/transformer
# classes for positional embedding and attention layers
def positional_encoding(length, depth):
  depth = depth/2

  positions = np.arange(length)[:, np.newaxis]     # (seq, 1)
  depths = np.arange(depth)[np.newaxis, :]/depth   # (1, depth)

  angle_rates = 1 / (10000**depths)         # (1, depth)
  angle_rads = positions * angle_rates      # (pos, depth)

  pos_encoding = np.concatenate(
      [np.sin(angle_rads), np.cos(angle_rads)],
      axis=-1)

  return tf.cast(pos_encoding, dtype=tf.float32)


class PositionalEmbedding(tf.keras.layers.Layer):
  def __init__(self, vocab_size, d_model):
    super().__init__()
    self.d_model = d_model
    self.embedding = tf.keras.layers.Embedding(vocab_size, d_model, mask_zero=True)
    self.pos_encoding = positional_encoding(length=2048, depth=d_model)

  def compute_mask(self, *args, **kwargs):
    return self.embedding.compute_mask(*args, **kwargs)

  def call(self, x):
    length = tf.shape(x)[1]
    x = self.embedding(x)
    # This factor sets the relative scale of the embedding and positonal_encoding.
    x *= tf.math.sqrt(tf.cast(self.d_model, tf.float32))
    x = x + self.pos_encoding[tf.newaxis, :length, :]
    return x


class BaseAttention(tf.keras.layers.Layer):
  def __init__(self, **kwargs):
    super().__init__()
    self.mha = tf.keras.layers.MultiHeadAttention(**kwargs)
    self.layernorm = tf.keras.layers.LayerNormalization()
    self.add = tf.keras.layers.Add()


class CausalSelfAttention(BaseAttention):
  def call(self, x):
    attn_output = self.mha(
        query=x,
        value=x,
        key=x,
        use_causal_mask = True)
    x = self.add([x, attn_output])
    x = self.layernorm(x)
    return x

In [3]:
# classes for NN model architecture
class FeedForward(tf.keras.layers.Layer):
  def __init__(self, d_model, dff, dropout_rate=0.1):
    super().__init__()
    self.seq = tf.keras.Sequential([
      tf.keras.layers.Dense(dff, activation='relu'),
      tf.keras.layers.Dense(d_model),
      tf.keras.layers.Dropout(dropout_rate)
    ])
    self.add = tf.keras.layers.Add()
    self.layer_norm = tf.keras.layers.LayerNormalization()

  def call(self, x):
    x = self.add([x, self.seq(x)])
    x = self.layer_norm(x)
    return x


class DecoderLayer(tf.keras.layers.Layer):
  def __init__(self, *, d_model, num_heads, dff, dropout_rate=0.1):
    super(DecoderLayer, self).__init__()

    self.causal_self_attention = CausalSelfAttention(
        num_heads=num_heads,
        key_dim=d_model,
        dropout=dropout_rate)

    self.ffn = FeedForward(d_model, dff)

    self.add = tf.keras.layers.Add()
    self.layer_norm = tf.keras.layers.LayerNormalization()

  def call(self, x):
    y = self.causal_self_attention(x=x)

    x = self.add([x, y])
    x = self.layer_norm(x)

    x = self.ffn(x)  # Shape `(batch_size, seq_len, d_model)`.
    return x


class Decoder(tf.keras.layers.Layer):
  def __init__(self, *, num_layers, d_model, num_heads, dff, vocab_size,
               dropout_rate=0.1):
    super(Decoder, self).__init__()

    self.d_model = d_model
    self.num_layers = num_layers

    self.pos_embedding = PositionalEmbedding(vocab_size=vocab_size,
                                             d_model=d_model)
    self.dropout = tf.keras.layers.Dropout(dropout_rate)
    self.dec_layers = [
        DecoderLayer(d_model=d_model, num_heads=num_heads,
                     dff=dff, dropout_rate=dropout_rate)
        for _ in range(num_layers)]

  def call(self, x):
    # `x` is token-IDs shape (batch, target_seq_len)
    x = self.pos_embedding(x)  # (batch_size, target_seq_len, d_model)

    x = self.dropout(x)

    for i in range(self.num_layers):
      x  = self.dec_layers[i](x)

    # The shape of x is (batch_size, target_seq_len, d_model).
    return x


class GPT(tf.keras.Model):
  def __init__(self, *, num_layers, d_model, num_heads, dff,
               input_vocab_size, target_vocab_size, dropout_rate=0.1):
    super().__init__()

    self.decoder = Decoder(num_layers=num_layers, d_model=d_model,
                           num_heads=num_heads, dff=dff,
                           vocab_size=target_vocab_size,
                           dropout_rate=dropout_rate)

    self.final_layer = tf.keras.layers.Dense(target_vocab_size)

  def call(self, x):
    # To use a Keras model with `.fit` you must pass all your inputs in the
    # first argument.

    x = self.decoder(x)  # (batch_size, target_len, d_model)

    # Final linear layer output.
    logits = self.final_layer(x)  # (batch_size, target_len, target_vocab_size)

    # Drop the keras mask, so it doesn't scale the losses/metrics.
    try:
      del logits._keras_mask
    except AttributeError:
      pass

    # Return the final output and the attention weights.
    return logits


class CustomSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):
  def __init__(self, d_model, warmup_steps=4000):
    super().__init__()

    self.d_model = d_model
    self.d_model = tf.cast(self.d_model, tf.float32)

    self.warmup_steps = warmup_steps

  def __call__(self, step):
    step = tf.cast(step, dtype=tf.float32)
    arg1 = tf.math.rsqrt(step)
    arg2 = step * (self.warmup_steps ** -1.5)

    return tf.math.rsqrt(self.d_model) * tf.math.minimum(arg1, arg2)


# performance metrics
def masked_loss(label, pred):
  mask = label != 0
  loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')
  loss = loss_object(label, pred)

  mask = tf.cast(mask, dtype=loss.dtype)
  loss *= mask

  loss = tf.reduce_sum(loss)/tf.reduce_sum(mask)
  return loss


def masked_accuracy(label, pred):
  pred = tf.argmax(pred, axis=2)
  label = tf.cast(label, pred.dtype)
  match = label == pred

  mask = label != 0

  match = match & mask

  match = tf.cast(match, dtype=tf.float32)
  mask = tf.cast(mask, dtype=tf.float32)
  return tf.reduce_sum(match)/tf.reduce_sum(mask)

In [4]:
train_data, val_data, tokenizer= PrepareDataset(
    corpus=corpus,
    n=125,
    batch_size=32,
    train_split=0.8
)

unique_words = len(tokenizer.word_index) + 1
d_model = 128

learning_rate = CustomSchedule(d_model)

optimizer = tf.keras.optimizers.Adam(
    learning_rate,
    beta_1=0.9,
    beta_2=0.98,
    epsilon=1e-9
)

num_layers = 8
dff = 512
num_heads = 8
dropout_rate = 0.1
epochs = 100

gpt = GPT(
    num_layers=num_layers,
    d_model=d_model,
    num_heads=num_heads,
    dff=dff,
    input_vocab_size=unique_words,
    target_vocab_size=unique_words,
    dropout_rate=dropout_rate
)

gpt.compile(
    loss=masked_loss,
    optimizer=optimizer,
    metrics=[masked_accuracy]
)

gpt.fit(train_data, validation_data=val_data, epochs=epochs)

Epoch 1/100
8/8 [==============================] - 36s 484ms/step - loss: 6.4148 - masked_accuracy: 0.0026 - val_loss: 6.4264 - val_masked_accuracy: 0.0017
Epoch 2/100
8/8 [==============================] - 1s 160ms/step - loss: 6.4006 - masked_accuracy: 0.0027 - val_loss: 6.3987 - val_masked_accuracy: 0.0026
Epoch 3/100
8/8 [==============================] - 1s 178ms/step - loss: 6.3558 - masked_accuracy: 0.0035 - val_loss: 6.3520 - val_masked_accuracy: 0.0049
Epoch 4/100
8/8 [==============================] - 1s 188ms/step - loss: 6.2968 - masked_accuracy: 0.0064 - val_loss: 6.2894 - val_masked_accuracy: 0.0114
Epoch 5/100
8/8 [==============================] - 2s 271ms/step - loss: 6.2244 - masked_accuracy: 0.0111 - val_loss: 6.2165 - val_masked_accuracy: 0.0204
Epoch 6/100
8/8 [==============================] - 2s 224ms/step - loss: 6.1411 - masked_accuracy: 0.0200 - val_loss: 6.1407 - val_masked_accuracy: 0.0298
Epoch 7/100
8/8 [==============================] - 1s 158ms/step - lo

In [5]:
# model the output
test_idx = 1

for input, labels in val_data.take(test_idx):
    break

test_labels = labels[0][12:-13]

output_array = tf.TensorArray(dtype=tf.int64, size=0, dynamic_size=True)

for i in range(len(test_labels.numpy())):
    output_array = output_array.write(i, test_labels.numpy()[i])

# output_array = output_array.write(0, 1)
# output_array = output_array.write(1, 100)

for i in tf.range(len(output_array.stack().numpy()), len(input[0])*2):
    output = tf.reshape(output_array.stack(), (1, len(output_array.stack())))
    predictions = gpt(output, training=False)

    # Select the last token from the `seq_len` dimension.
    predictions = predictions[:, -1:, :]  # Shape `(batch_size, 1, vocab_size)`.

    predicted_id = tf.argmax(predictions, axis=-1)[0][0]

    # Concatenate the `predicted_id` to the output which is given to the
    # decoder as its input.
    output_array = output_array.write(i+1, predicted_id)

    if predicted_id == 4:
        break

[tokenizer.index_word[s] for s in output_array.stack().numpy() if s != 0]

['funky-bitch',
 'maze',
 'ocelot',
 'sparkle',
 'cavern',
 'wingsuit',
 'set-2',
 'carini',
 'ghost',
 'mikes-song',
 'simple',
 'joy',
 'weekapaug-groove',
 'julius',
 'sand',
 'wading-in-the-velvet-sea',
 'you-enjoy-myself',
 'set-e',
 'quinn-the-eskimo-the-mighty-quinn',
 'eos',
 'set-1',
 'my-soul',
 'bathtub-gin',
 '555',
 'pebbles-and-marbles',
 'the-line',
 'vultures',
 'fast-enough-for-you',
 'back-on-the-train',
 'taste',
 'gumbo',
 'halfway-to-the-moon',
 'stealing-time-from-the-faulty-plan',
 'suzy-greenberg',
 'set-2',
 'chalk-dust-torture',
 'scents-and-subtle-sounds',
 'twist',
 'fuego',
 'the-wedge',
 'light',
 'harry-hood',
 'first-tube',
 'set-e',
 'fluffhead',
 'eos',
 'set-1',
 'llama',
 'undermind',
 'stash',
 'halfway-to-the-moon',
 'i-didnt-know',
 'nellie-kane',
 'guyute',
 'the-line',
 'ocelot',
 'no-quarter',
 'ha-ha-ha',
 'suzy-greenberg',
 'set-2',
 '46-days',
 'back-on-the-train',
 'simple',
 'ghost',
 'backwards-down-the-number-line',
 'harry-hood',
 'wadi

In [ ]:
print(test_labels)

print(output_array.stack())

tf.Tensor(
[ 62  25 110  27  11 197   2  85  49   9  51 167  10  42  80 108   8   3
 228   4   1 126  35 152 238 209 186 134  79  83  94 182 133  19   2   7
 198  70 131  89  93  21  92   3  52   4   1  30 170  16 182  54 136  87
 209 110 267 219  19   2  84  79  51  49  88  21 108  15   3  41   4   1
  48  56  82 133 152  33  31 261 223  27 197  17  11   2  28 158  85  93
 131  44 143 165   3  66  60  52   4], shape=(99,), dtype=int64)
tf.Tensor(
[ 62  25 110  27  11 197   2  85  49   9  51 167  10  42  80 108   8   3
 228   4   1 126  35 152 238 209 186 134  79  83  94 182 133  19   2   7
 198  70 131  89  93  21  92   3  52   4   1  30 170  16 182  54 136  87
 209 110 267 219  19   2  84  79  51  49  88  21 108  15   3  41   4   1
  48  56  82 133 152  33  31 261 223  27 197  17  11   2  28 158  85  93
 131  44 143 165   3  66  60  52   4   0   1 103  56  23  45 127  16  13
  16   2  28  65  57  70  57 108  21   3  77   4], shape=(120,), dtype=int64)


In [ ]:
[tokenizer.index_word[s] for s in labels[test_idx-1].numpy() if s != 0]

['set-e',
 'bouncing-around-the-room',
 'tweezer-reprise',
 'eos',
 'set-1',
 'acdc-bag',
 'poor-heart',
 'cities',
 'kill-devil-falls',
 'reba',
 'possum',
 'sample-in-a-jar',
 'funky-bitch',
 'maze',
 'ocelot',
 'sparkle',
 'cavern',
 'wingsuit',
 'set-2',
 'carini',
 'ghost',
 'mikes-song',
 'simple',
 'joy',
 'weekapaug-groove',
 'julius',
 'sand',
 'wading-in-the-velvet-sea',
 'you-enjoy-myself',
 'set-e',
 'quinn-the-eskimo-the-mighty-quinn',
 'eos',
 'set-1',
 'my-soul',
 'bathtub-gin',
 '555',
 'pebbles-and-marbles',
 'the-line',
 'vultures',
 'fast-enough-for-you',
 'back-on-the-train',
 'taste',
 'gumbo',
 'halfway-to-the-moon',
 'stealing-time-from-the-faulty-plan',
 'suzy-greenberg',
 'set-2',
 'chalk-dust-torture',
 'scents-and-subtle-sounds',
 'twist',
 'fuego',
 'the-wedge',
 'light',
 'harry-hood',
 'first-tube',
 'set-e',
 'fluffhead',
 'eos',
 'set-1',
 'llama',
 'undermind',
 'stash',
 'halfway-to-the-moon',
 'i-didnt-know',
 'nellie-kane',
 'guyute',
 'the-line',
 '

In [ ]:
input, labels = val_data.take(1500)

In [ ]:
labels[0][12:-13]


<tf.Tensor: shape=(5, 124), dtype=int64, numpy=
array([[187, 106, 188, 300, 263,   3, 236,  92,   4,   1,  98, 118, 109,
         45, 148, 325,  16, 194,  13, 111, 361,   2, 245, 281,  64, 176,
        129,  21,   3,  47,  38,   4,   1,   9,  91, 144,  10, 180, 395,
         89, 121, 378,  52,   2,  40,  14, 268,  57, 193,  70,  88,   3,
        247,  26,   4,   1,  84, 206, 161, 124, 223,  34, 186, 238,  85,
          2, 269, 190,   7, 300,  93, 218,   7,  18,  44,   3,  11, 107,
         72,   4,   1, 172, 237, 172, 280, 137, 306,  49,  58, 170,  61,
         17,   2, 188, 131, 128, 210, 181, 246,   8,   3, 165,   4,   1,
         56, 147,  80, 231, 286,  20,  35,   2, 202, 282,  51, 213,  51,
        143, 240,   6, 151, 313,  21,   3],
       [211,   4,   1, 281,  21, 308,  24, 152, 235, 177, 210,  41,   2,
        198, 218,  48, 151,  57,  49,   6, 231,  61, 154, 139,  22,   3,
         80,   4,   1,  12,  82,  84,  44,  51, 239, 244, 183,  15,   2,
         35, 188, 227,  89,  56,

In [6]:
def generate_sequence(seed_sequence, max_length=25):
    encoder_input = tf.constant([tokenizer.word_index[word] for word in seed_sequence], dtype=tf.int64)
    encoder_input = tf.reshape(encoder_input, (1, len(encoder_input)))

    output_array = tf.TensorArray(dtype=tf.int64, size=0, dynamic_size=True)
    output_array = output_array.write(0, tokenizer.word_index[seed_sequence[-1]])

    unique_tokens = set([tokenizer.word_index[word] for word in seed_sequence])

    i = 0
    while i < max_length:
        output = tf.reshape(output_array.stack(), (1, len(output_array.stack())))
        predictions = gpt(output, training=False)

        predictions = predictions[:, -1:, :]
        predicted_id = tf.argmax(predictions, axis=-1)[0][0]

        # Convert the scalar TensorFlow tensor to a tuple
        predicted_id_tuple = (predicted_id.numpy(),)

        # Check for duplicates
        if predicted_id_tuple in unique_tokens :

            output_array = output_array.write(i+1, predicted_id)
            i += 1
            continue

        unique_tokens.add(predicted_id_tuple)

        output_array = output_array.write(i+1, predicted_id)

        unique_tokens = set(unique_tokens)
        #if predicted_id == 4:
        #    output_array = output_array.write(i+1, predicted_id)
        #    i += 1
        #    continue;

        i += 1

    predicted_sequence = [tokenizer.index_word[s] for s in output_array.stack().numpy()]

    return predicted_sequence

# Example usage:
#org_list = corpus[-1].split()
#org_list = ['sample-in-a-jar', 'sand', '555', 'rift', 'halfway-to-the-moon', 'horn', 'devotion-to-a-dream', 'blaze-on', 'tube', 'wolfmans-brother', 'set-2', 'ghost', 'birds-of-a-feather', 'mikes-song', 'the-wedge', 'fuego', 'shade', 'no-men-in-no-mans-land', 'weekapaug-groove', 'boogie-on-reggae-woman', 'chalk-dust-torture', 'set-e', 'theme-from-the-bottom', 'eos']
org_list = ['farmhouse', 'first-tube', 'twist', 'divided-sky', 'ginseng-sullivan', 'carini', 'whats-the-use', 'wildcard', 'set-2', 'down-with-disease', 'the-moma-dance', 'piper', 'fee', 'gotta-jibboo', 'saw-it-again', 'split-open-and-melt', 'cavern', 'david-bowie', 'set-e', 'the-squirming-coil', 'eos']
seed_sequence = org_list[:3]
max_length = len(org_list) - len(seed_sequence)
predicted_sequence = generate_sequence(seed_sequence, max_length=max_length)

print("Original Sequence:", org_list)
print("Seed Sequence:", seed_sequence)
print("Predicted Sequence:", (predicted_sequence[len(seed_sequence):]))


Original Sequence: ['farmhouse', 'first-tube', 'twist', 'divided-sky', 'ginseng-sullivan', 'carini', 'whats-the-use', 'wildcard', 'set-2', 'down-with-disease', 'the-moma-dance', 'piper', 'fee', 'gotta-jibboo', 'saw-it-again', 'split-open-and-melt', 'cavern', 'david-bowie', 'set-e', 'the-squirming-coil', 'eos']
Seed Sequence: ['farmhouse', 'first-tube', 'twist']
Predicted Sequence: ['set-e', 'loving-cup', 'eos', 'set-1', 'acdc-bag', 'the-moma-dance', 'runaway-jim', 'the-moma-dance', 'funky-bitch', 'the-moma-dance', 'funky-bitch', 'the-moma-dance', 'ocelot', 'set-2', 'down-with-disease', 'piper']


In [7]:

stopword_list = {}
# Convert the lists to sets
org_set = set(org_list)
seed_set = set(seed_sequence)
not_in_seed_sequence = org_set - seed_set
filtered_songs = [song.strip() for song in not_in_seed_sequence if song not in stopword_list]
predicted_sequence = [song.strip() for song in predicted_sequence[len(seed_sequence):] if song not in stopword_list]

# Count the number of matching songs
matching_songs = [song for song in filtered_songs if song in predicted_sequence]
num_matching_songs = len(matching_songs)

# Calculate the percentage of matching songs
percentage_matching = (num_matching_songs / len(filtered_songs)) * 100

print("Matching Songs in Predicted Sequence:")
print(matching_songs)
print("Number of Matching Songs:", num_matching_songs)
print("Percentage of Matching Songs:", percentage_matching)

Matching Songs in Predicted Sequence:
['set-2', 'down-with-disease', 'set-e', 'piper', 'eos', 'the-moma-dance']
Number of Matching Songs: 6
Percentage of Matching Songs: 33.33333333333333
